# Crear el dominio del glaciar 

En este cuaderno aprenderemos a crear el dominio del glaciar. Usaremos los netcdf creados previamente. 

## Importar librerias **`Xarray`** y **`Numpy`** 

In [ ]:
# Importar las bibliotecas necesarias
# Xarray: Para manejar datos en formato NetCDF
# NumPy: Para operaciones numéricas
import xarray as xr
import numpy as np

In [ ]:
# Cargar los datos del modelo de elevación digital (DEM) y la máscara desde archivos NetCDF
dem  = xr.open_dataset('./dem_pros/Bell_dem_fill.nc')
mask = xr.open_dataset('./dem_pros/Bell_mask.nc')
slope = xr.open_dataset('./dem_pros/Bell_slope.nc')
aspect = xr.open_dataset('./dem_pros/Bell_aspect.nc')

In [ ]:
dem.Band1.plot()

In [ ]:
mask.Band1.plot()

In [ ]:
mask=mask.Band1.values
mask[np.isnan(mask)]=0
mask[mask>0]=1

In [ ]:
ds = xr.Dataset()
ds

In [ ]:
ds.coords['lon'] = dem.lon.values
ds.lon.attrs['standard_name'] = 'lon'
ds.lon.attrs['long_name'] = 'longitude'
ds.lon.attrs['units'] = 'degrees_east'

ds.coords['lat'] = dem.lat.values
ds.lat.attrs['standard_name'] = 'lat'
ds.lat.attrs['long_name'] = 'latitude'
ds.lat.attrs['units'] = 'degrees_north'

In [ ]:
ds

In [ ]:
### Función para insertar variables en el conjunto de datos
def insert_var(ds, var, name, units, long_name):
    ds[name] = (('lat','lon'), var)
    ds[name].attrs['units'] = units
    ds[name].attrs['long_name'] = long_name
    ds[name].attrs['_FillValue'] = -9999

In [ ]:
### Insertar las variables estáticas necesarias
insert_var(ds, dem.Band1.values,'HGT','meters','meter above sea level')
insert_var(ds, aspect.Band1.values,'ASPECT','degrees','Aspect of slope')
insert_var(ds, slope.Band1.values,'SLOPE','degrees','Terrain slope')
insert_var(ds, mask,'MASK','boolean','Glacier mask')

In [ ]:
ds

In [ ]:
print(ds.lon.min(), ds.lon.max())
print(ds.lat.min(), ds.lat.max())

In [ ]:
### Mostrar el número de puntos de cuadrícula de los glaciares
print("Study area consists of ", np.nansum(mask[mask==1]), " glacier points")

In [ ]:
### Guardar el archivo estático combinado
ds.to_netcdf('./dom/Bell_dom.nc')

In [ ]:
ds.SLOPE.plot()